# Gestructureerde data: inladen dataset

## Enkel datatype

Er zijn heel veel verschillende bronnen om datasets te vinden. Je kan ze bijvoorbeeld op keras, overheidssites en dergelijke vinden.
Ook zijn er heel wat datasets die standaard geworden zijn voor voorbeelden/modellen te testen. 
Deze datasets worden mee aangeleverd met de verschillende modelling-frameworks en kunnen zo eenvoudig gebruikt worden zonder een extra download uit te voeren.
In deze notebook gaan we werken met deze standaard datasets.

In het eerste voorbeeld gaan we werken met een dataset waarbij alle data reeds numeriek is, meer bepaald gaan we werken met de iris-dataset dat informatie over de bloemen van een aantal iris-soorten bevat.

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train

array([[6.3, 2.5, 5. , 1.9],
       [5.9, 3.2, 4.8, 1.8],
       [5.5, 2.4, 3.8, 1.1],
       [5.2, 2.7, 3.9, 1.4],
       [5.1, 3.3, 1.7, 0.5],
       [6.5, 3. , 5.8, 2.2],
       [5.8, 2.7, 5.1, 1.9],
       [5.4, 3.4, 1.7, 0.2],
       [6.1, 2.8, 4. , 1.3],
       [4.8, 3.1, 1.6, 0.2],
       [4.5, 2.3, 1.3, 0.3],
       [6.5, 3. , 5.2, 2. ],
       [6.3, 2.9, 5.6, 1.8],
       [6.8, 3.2, 5.9, 2.3],
       [6. , 2.7, 5.1, 1.6],
       [6.6, 2.9, 4.6, 1.3],
       [5.8, 2.6, 4. , 1.2],
       [6.7, 3.1, 4.4, 1.4],
       [6.6, 3. , 4.4, 1.4],
       [6.9, 3.2, 5.7, 2.3],
       [5.1, 3.8, 1.5, 0.3],
       [6.3, 2.5, 4.9, 1.5],
       [7.7, 2.8, 6.7, 2. ],
       [5.7, 3.8, 1.7, 0.3],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3.3, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [6. , 3. , 4.8, 1.8],
       [6. , 2.2, 5. , 1.5],
       [7.2, 3.2, 6. , 1.8],
       [5. , 3.4, 1.6, 0.4],
       [6.2, 2.2, 4.5, 1.5],
       [6.7, 3. , 5. , 1.7],
       [5.5, 4

### Pytorch

Om een dataset in lezen en klaar te maken in pytorch moet je werken met een Dataset. Dit kan je doen als volgt:

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

# dit bevat de preprocessing stappen dus heel sterk afhankelijk van de dataset waarmee je werkt
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # deze functie return de rij met index idx
        return self.X[idx], self.y[idx] # return inputs, labels

train_dataset = IrisDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

### Keras

Keras is vooral bedoeld als uitvoerend framework en niet om data in te laden.

## Mixed types

Bovenstaande voorbeelden zijn eenvoudige voorbeelden omdat de data in principe reeds gepreprocessed is of uit slechts 1 datatype bestaat. 
Dit heeft als gevolg dat alle inputs op dezelfde manier verwerkt kunnen worden en er dus geen onderscheid gemaakt moet worden tussen verschillende kolommen.
Data met hetzelfde type kan dus eenvoudig aan een sequentieel neuraal netwerk model gepresenteerd worden.

Indien de dataset echter meerdere types bevatten kan de data niet rechtstreeks aan het neuraal netwerk doorgegeven worden aangezien dit type model enkel met numerieke data kan werken. 
Een standaard dataset waarin dit gebeurd is de titanic dataset.
Hoe dit gebeurd in de verschillende frameworks zie je hieronder

### Pytorch

Aangezien we data binnenkrijgen in een dataset als numpy array/dataframe kunnen we in de constructor of de get_item functie de nodige preprocessing stappen uitvoeren.
Hieronder doe ik het rechtstreeks met pandas, je kan het ook met een sci-kit learn pipeline doen.

**Let op:** Hieronder splits ik niet op train en testdata dus kan ik de preprocessingen stappen in de constructor doen van de dataset.
Indien er echter eerst een splitsing gebeurt in train- en testdata, dan moet je de preprocessing stappen in de test-data doen met de parameters geleerd uit de dataset.
Dit is bijvoorbeeld belangrijk als er een klasse niet aanwezig is in de dataset. Dan zou deze niet gemapt worden en komt de ordinal encoding niet overeen tussen train- en testdata.

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Load Titanic dataset
titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")

# dit bevat de preprocessing stappen dus heel sterk afhankelijk van de dataset waarmee je werkt
class TitanicDataset(Dataset):
    def __init__(self, dataframe):
        # null-waarden invullen
        dataframe.fillna({'age': dataframe['age'].median()}, inplace=True)
        dataframe.fillna({'embark_town': dataframe['embark_town'].mode()[0]}, inplace=True)
        # ordinal encoding
        dataframe['sex'] = dataframe['sex'].astype('category').cat.codes
        dataframe['class'] = dataframe['class'].astype('category').cat.codes
        dataframe['embark_town'] = dataframe['embark_town'].astype('category').cat.codes

        X = dataframe['class', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare']
        y = dataframe['survived']

        self.X = torch.tensor(X_values, dtype=torch.float32)
        self.y = torch.tensor(y_values, dtype=torch.long)
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # deze functie return de rij met index idx
        return self.X[idx], self.y[idx] # return inputs, labels

dataset = TitanicDataset(titanic) # hier moet je opletten, want hier is eigenlijk data leakage aanwezig (de mediaan)

# nog splitsen in train_test
train_size = int(0.8 *len(dataset))
test_size = len(dataset)-train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])